#### Naive Bayes

In [2]:
import os
import re
import json
import gzip
import wget
import pandas as pd
import numpy as np
from urllib.request import urlopen
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from prettytable import PrettyTable
import time
import nltk
from imp import reload

#cleaning textfiles libraries
from collections import defaultdict # For accumlating values
from nltk.corpus import stopwords # To remove stopwords

C:\Users\Triston\AppData\Local\Temp\ipykernel_18740\992867872.py:15: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload


In [ ]:
##download data from url
### randomly selected file to model
#url = 'https://jmcauley.ucsd.edu/data/amazon_v2/categoryFilesSmall/Grocery_and_Gourmet_Food_5.json.gz'
#filename = wget.download(url)


# read JSON string from file and convert to dataframe
#df = pd.read_json('C:/Users/Triston/OneDrive/Documents/DS6120 Capstone A/Grocery_and_Gourmet_Food_5-CLEAN.json', compression='gzip')

In [3]:
#load metadata
data = []
with gzip.open('C:/Users/Triston/OneDrive/Documents/DS6120 Capstone A/Grocery_and_Gourmet_Food_5-CLEAN.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(data))

# first row of the list
print(data[0])

969400
{'overall': 5.0, 'verified': True, 'reviewTime': '11 19, 2014', 'reviewerID': 'A1QVBUH9E1V6I8', 'asin': '4639725183', 'reviewerName': 'Jamshed Mathur', 'reviewText': 'No adverse comment.', 'unixReviewTime': 1416355200, 'vote': 0.0, 'helpful_flag': 0, 'totalWords': 3, 'vocab_size': 3, 'char_counts': 17, 'avg_word_size': 5.6666666667, 'stopword_count': 0, 'numeric_count': 0, 'UpperCase_word_count': 0, 'Capitalized_word_count': 1, 'Nouns_count': 1, 'Verbs_count': 0, 'Adj_count': 1, 'Adverbs_count': 0, 'Conj_count': 0, 'binned': {'closed': 'right', 'closed_left': False, 'closed_right': True, 'is_empty': False, 'left': -1, 'length': 1, 'mid': -0.5, 'open_left': True, 'open_right': False, 'right': 0}}


In [4]:
# convert list into pandas dataframe

df = pd.DataFrame.from_dict(data)

print(len(df))

969400


In [5]:
#look at dataframe
df.info()
display(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 969400 entries, 0 to 969399
Data columns (total 24 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   overall                 969400 non-null  float64
 1   verified                969400 non-null  bool   
 2   reviewTime              969400 non-null  object 
 3   reviewerID              969400 non-null  object 
 4   asin                    969400 non-null  object 
 5   reviewerName            969293 non-null  object 
 6   reviewText              969400 non-null  object 
 7   unixReviewTime          969400 non-null  int64  
 8   vote                    969400 non-null  float64
 9   helpful_flag            969400 non-null  int64  
 10  totalWords              969400 non-null  int64  
 11  vocab_size              969400 non-null  int64  
 12  char_counts             969400 non-null  int64  
 13  avg_word_size           969400 non-null  float64
 14  stopword_count      

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,unixReviewTime,vote,helpful_flag,...,stopword_count,numeric_count,UpperCase_word_count,Capitalized_word_count,Nouns_count,Verbs_count,Adj_count,Adverbs_count,Conj_count,binned
0,5.0,True,"11 19, 2014",A1QVBUH9E1V6I8,4639725183,Jamshed Mathur,No adverse comment.,1416355200,0.0,0,...,0,0,0,1,1,0,1,0,0,"{'closed': 'right', 'closed_left': False, 'clo..."
1,5.0,True,"10 13, 2016",A3GEOILWLK86XM,4639725183,itsjustme,Gift for college student.,1476316800,0.0,0,...,1,0,0,1,3,0,0,0,1,"{'closed': 'right', 'closed_left': False, 'clo..."
2,5.0,True,"11 21, 2015",A32RD6L701BIGP,4639725183,Krystal Clifton,"If you like strong tea, this is for you. It mi...",1448064000,0.0,0,...,8,0,0,2,1,3,3,2,4,"{'closed': 'right', 'closed_left': False, 'clo..."
3,5.0,True,"08 12, 2015",A2UY1O1FBGKIE6,4639725183,U. Kane,Love the tea. The flavor is way better than th...,1439337600,0.0,0,...,5,0,0,4,7,2,2,1,2,"{'closed': 'right', 'closed_left': False, 'clo..."
4,5.0,True,"05 28, 2015",A3QHVBQYDV7Z6U,4639725183,The Nana,I have searched everywhere until I browsed Ama...,1432771200,0.0,0,...,13,0,4,8,5,10,5,4,7,"{'closed': 'right', 'closed_left': False, 'clo..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969395,5.0,True,"09 8, 2017",A223YRQH2Z5T1D,B01HJF6FRA,flint5292,"As a new vegan, it is sometimes difficult to r...",1504828800,4.0,1,...,29,0,2,9,20,15,11,6,10,"{'closed': 'right', 'closed_left': False, 'clo..."
969396,5.0,True,"08 4, 2017",A38GDA4TB9EILT,B01HJF6FRA,Moriah Bolyard,The best thing ever is ordering a product you ...,1501804800,3.0,1,...,17,0,3,8,16,8,2,3,6,"{'closed': 'right', 'closed_left': False, 'clo..."
969397,5.0,True,"07 4, 2017",A2025PN7HDC5BO,B01HJF6FRA,M.C,I used to love ranch before I became vegan. It...,1499126400,5.0,1,...,70,0,8,19,49,36,14,10,30,"{'closed': 'right', 'closed_left': False, 'clo..."
969398,5.0,True,"06 7, 2017",A1NY7XWC7EPQOA,B01HJF6FRA,Greensboro,I cannot have dairy nor gluten. This is as cl...,1496793600,2.0,1,...,13,0,2,4,1,8,3,2,4,"{'closed': 'right', 'closed_left': False, 'clo..."


In [7]:
#convert column to string
df["reviewText"]=df["reviewText"].astype(str)

In [8]:
#Determine Average Review Length and add review length column to dataframe

x = [len(df['reviewText'][i]) for i in range(df['reviewText'].shape[0])]
print('average length of review: {:.3f}'.format(sum(x)/len(x)) )

average length of review: 218.082
